# Installs (do this only once)

In [ ]:
import Pkg; 
Pkg.add("DataStructures")
Pkg.add("Graphs")
Pkg.add("Plots")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("GraphPlot")


# Imports

In [ ]:
using Graphs, GraphPlot
using DataStructures
using Statistics
using Plots
using Random
using CSV
using DataFrames
using Graphs.SimpleGraphs

# Compute Counts

In [ ]:
function compute_edgecounts(g::SimpleGraph{Int64},  minimize::Bool)::Vector{Int64}
    pq = PriorityQueue{Int64, Int64}()
    si_edges_set = Set{Tuple{Int64, Int64}}([])
    si_counts_record = Vector{Int64}()
    already_infected = Set{Int64}([])
    direction = minimize ?  1 : -1
    
    # setup
    # the priorty encodes how many S-I eges will added (in total) if node turns to S
    # Some S-I edges will also be removed (turn to S-S) edges
    for (v_i, degree) in enumerate(degree(g))  
        enqueue!(pq, v_i, direction * degree)
    end
    append!(si_counts_record, length(si_edges_set))  # zero S-I edges in the beginning
    
    # greedy 
    while length(pq) > 0
        v_i = dequeue!(pq)
        push!(already_infected, v_i)

        for v_j in neighbors(g, v_i)
            edge = (min(v_j, v_i), max(v_j, v_i))
            if v_j in already_infected
                delete!(si_edges_set, edge) # this turns form S-I to an S-S edge
            else
                push!(si_edges_set, edge)  # this turns form I-I to an S-I edge
                delete!(pq, v_j)      
                neig_j =  neighbors(g, v_j)
                neighbors_list_susc = filter(v_x -> !(v_x in already_infected), neig_j)
                num_sus_neighbors = length(neighbors_list_susc)
                num_inf_neighbors = length(neig_j) - num_sus_neighbors
                d_j = num_sus_neighbors - num_inf_neighbors
                pq[v_j] = direction *  d_j
            end
        end

        append!(si_counts_record, length(si_edges_set))
    end
    
    # use symmetry 
    sym =  minimize ? min : max
    si_counts_record =  map((i,j)->sym(i,j), si_counts_record, reverse(si_counts_record))
    return si_counts_record
end

In [ ]:
function compute_edgecounts_minmax(g)
    counts_min = compute_edgecounts(g, true)
    counts_max = compute_edgecounts(g, false)
    return(counts_min, counts_max)
end

### Test

In [ ]:
n = 100
g = erdos_renyi(n, 1/10, seed=123)
c_min, c_max = compute_edgecounts_minmax(g)
plot(c_min)
plot!(c_max)

# CTMC

In [ ]:
function solve_eq(edge_count_vec::Vector{Int64}, rate::Float64)::Vector{Float64}
    pi_hat::Vector{Float64}  = [log(1.0)]
    for (i, edge_count_i) in enumerate(edge_count_vec) 
        if (i == 1 && edge_count_i == 0)  || (i == length(edge_count_vec) && edge_count_i == 0)
            continue
        end
        inflow_from_left = rate*edge_count_vec[i]
        outflow_to_left = float(i+1)
        pi_hat_i = pi_hat[i-1] + log(inflow_from_left) - log(outflow_to_left)
        append!(pi_hat, pi_hat_i)
    end
    
    pi_hat = pi_hat .- maximum(pi_hat)
    pi_hat = map((x) -> exp(x), pi_hat)
    pi = pi_hat ./ sum(pi_hat)
    return pi
end

### test

In [ ]:
g = erdos_renyi(100, 1/20, seed=123)
c_min, c_max = compute_edgecounts_minmax(g)
eq_min = solve_eq(c_min, 1.0)
eq_max = solve_eq(c_max, 1.0)
plot(eq_min)
plot!(eq_max)

# Footprint

In [ ]:
function comput_mean(pi)
    mean = 0.0
    for (num_inf, prob) in enumerate(pi) 
        mean += num_inf*prob
    end
    return mean/length(pi)
end
    

function get_footprint(g::SimpleGraph{Int64}, rates, df)::Tuple{Vector{Float64},Vector{Float64}}
    footprint_min = si_counts_record = Vector{Float64}()
    footprint_max = si_counts_record = Vector{Float64}()
    c_min, c_max = compute_edgecounts_minmax(g)
    
    for rate in rates
        eq_min = solve_eq(c_min, rate)
        eq_max = solve_eq(c_max, rate)
        exp_mean_min = comput_mean(eq_min)
        exp_mean_max = comput_mean(eq_max)
        append!(footprint_min, exp_mean_min)
        append!(footprint_max, exp_mean_max)
        push!(df, (rates=rate, exp_inf_frac=exp_mean_min, bound="lower"), cols=:intersect)
        push!(df, (rates=rate, exp_inf_frac=exp_mean_max, bound="upper"), cols=:intersect)
    end
    return footprint_min, footprint_max
end

# Simlation (unused bc slow)

In [ ]:
function simulate_sis(g::SimpleGraph{Int64},  infection_rate::Float64, step_num::Int64=10^2)::Float64
    init_infected_num = Int64(nv(g)/2)
    init_infected = randperm(nv(g))[1:init_infected_num]
    
    currently_infected = Set{Int64}(init_infected)
    global_clock::Float64 = 0.0
    
    recovery_rate::Float64 = 1.0
    eps::Float64 = 0.000000000001 
    step_count::Int64 = 0
    
    for i in 1:step_num 
        min_j::Int64 = -1
        min_fire_time::Float64  = -1.0
        for v_j in 1:nv(g)
            rate::Float64 = v_j in currently_infected ? recovery_rate : infection_rate *  count(v_x->v_x in currently_infected, neighbors(g, v_j))
            rate += eps
            fire_time::Float64 = - log(rand())/rate
            if min_fire_time < 0   || fire_time < min_fire_time
                min_j = v_j
                min_fire_time = fire_time
            end
        end
        if !(min_j in currently_infected)
            push!(currently_infected, min_j)
        elseif length(currently_infected) > 1
            delete!(currently_infected, min_j)
        end
    end
    
    return length(currently_infected)/nv(g)
end

In [ ]:
function get_footprint_empirical_simple(g::SimpleGraph{Int64}, rates, df)
    footprint_values::Vector{Float64}  = []
    timepoints = range(0.0, stop=109.0, length=100)
     for rate in rates
        final_inf_frac::Vector{Float64}  = []
        for i in 1:20
            traj_end = simulate_sis(g, rate, nv(g)*100)
            print(".")
            append!(final_inf_frac, traj_end)
            push!(df, (rates=rate, exp_inf_frac=traj_end, bound="simulation"), cols=:intersect)
        end
        append!(footprint_values, mean(final_inf_frac))
    end
    return footprint_values, df
end


# Simulation (extern)

In [ ]:
function read_simout(filepath)
    #s = read("/Users/gerritgrossmann/Documents/devel/birth_death_palermo/simulation_output/simulation_10002842_1.txt", String)
    s = read(filepath, String)
    relevant_line =  "I," == split(s,"\n")[end-1][1:2] ? split(s,"\n")[end-1] : split(s,"\n")[end-2]
    value = split(relevant_line, ",")[2]
    value =  parse(Float64,  value) 
    return value
end

function simulate_sis_rust(g::SimpleGraph{Int64},  infection_rate::Float64, sim_num::Int64)#::Float64
    if !isdir("graph_output")
        run(`mkdir graph_output`)
    end 
    if !isdir("simulation_output")
        run(`mkdir simulation_output`)
    end 
    if !isdir("output")
        run(`mkdir output`)
    end 
        
    
    #infection_rate_str = length(repr(infection_rate)) > 7 ? repr(infection_rate)[:5] : repr(infection_rate)
    r = Int(floor(rand()*100000+10000000)) 
    #/Users/gerritgrossmann/Documents/devel/birth_death_palermo/graph_output
    graph_path = pwd() * "/graph_output/graph_export_$r.txt" #sorry
    binary_path = "/rust/Rejection-Based-Epidemic-Simulation/rust_reject/target/release/rust_reject"

    
    open("graph_output/graph_export_$r.txt", "w") do io
        for v_i in 1:nv(g) 
            v_j = v_i-1
            node_label = rand() > 0.5 ? "I" : "S"
            endchar = v_i != nv(g)  ? "\n" : "" 
            neighbors_corrected = map(x->x-1, neighbors(g,v_i))
            neig_str = replace(repr(neighbors_corrected), " " => "", "[" => "", "]" => "")
            write(io, repr(v_j) * ";" * node_label * ";" * neig_str *endchar)
        end
    end
    
    
    run_command = ""
    
    
    final_exp_inf::Vector{Float64} = []
    for i in 1:sim_num
        sim_path = pwd() * "/simulation_output/simulation_$r"*"_$i.txt"
        cmd = ".$binary_path $graph_path $sim_path $infection_rate"
        #cmd = """sh -c "$cmd" >> rust_output.txt 2>&1"""
        run(`sh -c $cmd \>\> rust_output.txt 2\>\&1`)
        sleep(0.1)
        v = read_simout(sim_path)
        push!(final_exp_inf, v)
    end 
    println(final_exp_inf)
    

    return final_exp_inf
end



function get_footprint_rust(g::SimpleGraph{Int64}, rates, df; sample_num=30)
   mean_agg::Vector{Float64} = []
    for infection_rate in rates
        print(".")
        mean_vec = simulate_sis_rust(g,  infection_rate, sample_num)
        for mean_v in mean_vec
            push!(df, (rates=infection_rate, exp_inf_frac=mean_v, bound="simulation"), cols=:intersect)
        end
        push!(mean_agg, mean(mean_vec))
    end
    return mean_agg
end

In [ ]:
n = 100
g = barabasi_albert(n, 3, seed=123)
simulate_sis_rust(g, 0.6, 10)

# Experiments

## n = 10^2

### Erdos

In [ ]:
n = 100
g = erdos_renyi(n, 1/n*10, seed=123)
START = 0.001
STOP = 10.0

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])
println("mean degree: ", 2*ne(g)/nv(g))

rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]

foot_min, foot_max = get_footprint(g, rates, df)
CSV.write("output/ERgraph_100.csv", df)

plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 

savefig("ERgraph_100.pdf") 





In [ ]:
rates_log = range(log(START), stop=log(STOP), length=100)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df, sample_num=100)
CSV.write("output/ERgraph_100.csv", df)
plot!(rates, foot_emp,  xaxis=:log) 


savefig("ERgraph_100_sim.pdf") 

### BA

In [ ]:
n = 100
g = barabasi_albert(n, 3, seed=123)
START = 0.001
STOP = 10.0

rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]
println("mean degree: ", 2*ne(g)/nv(g))

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])

foot_min, foot_max = get_footprint(g, rates, df)
CSV.write("output/BAgraph_100.csv", df)

plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 


savefig("BAgraph_100.pdf") 





In [ ]:
rates_log = range(log(START), stop=log(STOP), length=100)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df, sample_num=100)
CSV.write("output/BAgraph_100.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

savefig("BAgraph_100_sim.pdf") 

### WS

In [ ]:
n = 100
g = watts_strogatz(n, 8, 0.8, seed=123)
START = 0.001
STOP = 10.0

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])

rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]
println("mean degree: ", 2*ne(g)/nv(g))

foot_min, foot_max = get_footprint(g, rates, df)


CSV.write("output/WSgraph_100.csv", df)
plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 

savefig("WSgraph_100.pdf") 




In [ ]:
rates_log = range(log(START), stop=log(STOP), length=100)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df, sample_num=100)
CSV.write("output/WSgraph_100.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

savefig("WSgraph_100_sim.pdf") 

## n = 10^4

In [ ]:
# erdos
n = 10^4
g = erdos_renyi(n, 1/n*20, seed=123)
START = 0.001
STOP = 10.0

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])
println("mean degree: ", 2*ne(g)/nv(g))

rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]

foot_min, foot_max = get_footprint(g, rates, df)

CSV.write("output/ERgraph_10000.csv", df)

plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 




In [ ]:
rates_log = range(log(START), stop=log(STOP), length=100)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df)
CSV.write("output/ERgraph_10000.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

In [ ]:
# ba
n = 10^4
g = barabasi_albert(n, 4, seed=123)
println("mean degree: ", 2*ne(g)/nv(g))
START = 0.001
STOP = 10.0
rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])

foot_min, foot_max = get_footprint(g, rates, df)

CSV.write("output/BAgraph_10000.csv", df)
plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 



In [ ]:
rates_log = range(log(START), stop=log(STOP), length=100)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df)
CSV.write("output/BAgraph_10000.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

In [ ]:
# ws
n = 10^4
g = watts_strogatz(n, 12, 0.8, seed=123)
START = 0.001
STOP = 10.0
rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]
println("mean degree: ", 2*ne(g)/nv(g))

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])

foot_min, foot_max = get_footprint(g, rates, df)
CSV.write("output/WSgraph_10000.csv", df)
plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 



In [ ]:
rates_log = range(log(START), stop=log(STOP), length=100)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df)
CSV.write("output/WSgraph_10000.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

## n = 10^6

In [ ]:
# erdos
n = 10^6
g = erdos_renyi(n, 1/n*40, seed=123)
START = 0.001
STOP = 10.0
rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])
println("mean degree: ", 2*ne(g)/nv(g))

rates = range(START, stop=STOP, length=1000)
foot_min, foot_max = get_footprint(g, rates, df)

CSV.write("output/ERgraph_1000000.csv", df)
plot(rates, foot_min,  xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 



In [ ]:
rates_log = range(log(START), stop=log(STOP), length=20)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df, sample_num=5)
CSV.write("output/ERgraph_1000000.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

In [ ]:
# ba
n = 10^6
g = barabasi_albert(n, 6, seed=123)
START = 0.001
STOP = 10.0
rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])
println("mean degree: ", 2*ne(g)/nv(g))

foot_min, foot_max = get_footprint(g, rates, df)

CSV.write("output/BAgraph_1000000.csv", df)
plot(rates, foot_min, xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 



In [ ]:
rates_log = range(log(START), stop=log(STOP), length=20)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df, sample_num=5)
CSV.write("output/BAgraph_1000000.csv", df)
plot!(rates, foot_emp, xaxis=:log) 

In [ ]:
# ws
n = 10^6
g = watts_strogatz(n, 16, 0.8, seed=123)
START = 0.001
STOP = 10.0
rates_log = range(log(START), stop=log(STOP), length=1000)
rates = [exp(r) for r in rates_log]

df = DataFrame(rates = [], exp_inf_frac = [], bound=[])
println("mean degree: ", 2*ne(g)/nv(g))

foot_min, foot_max = get_footprint(g, rates, df)

CSV.write("output/WSgraph_1000000.csv", df)
plot(rates, foot_min, xaxis=:log) 
plot!(rates, foot_max, xaxis=:log) 



In [ ]:
rates_log = range(log(START), stop=log(STOP), length=20)
rates = [exp(r) for r in rates_log]

foot_emp = get_footprint_rust(g, rates, df, sample_num=5)
CSV.write("output/WSgraph_1000000.csv", df)
plot!(rates, foot_emp, xaxis=:log) 